In [1]:
import pyspark.sql.functions as F
from pyspark.sql import types as tp
from pyspark.sql import SparkSession
from pyspark.ml.regression import DecisionTreeRegressor, RandomForestRegressor, LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline, Transformer

In [2]:
spark = SparkSession.builder.config("spark.sql.shuffle.partitions", "200").config("spark.executor.memory", "4g").config("master", "yarn").getOrCreate()
spark

In [3]:
train = spark.read.csv("ml/train.csv",header = True, inferSchema=True)
test = spark.read.csv("ml/test.csv",header = True, inferSchema=True)

In [4]:
train.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



## 1. Average Purchase amount?

In [5]:
train.agg(F.round(F.sum("Purchase")/F.count("Purchase"),3).alias("avg_prchase_Amount"))

avg_prchase_Amount
9263.969


## 2. Counting and Removing null values

In [6]:
for i in train.columns:
    count_nulls = train.filter(F.col(i).isNull()).count()
    print(i,count_nulls)

User_ID 0
Product_ID 0
Gender 0
Age 0
Occupation 0
City_Category 0
Stay_In_Current_City_Years 0
Marital_Status 0
Product_Category_1 0
Product_Category_2 173638
Product_Category_3 383247
Purchase 0


In [7]:
train = train.dropna(subset = ["Product_Category_2","Product_Category_3"])

## 3. How many distinct values per column?

In [10]:
for i in train.columns[2:]:
    distinct_counts  = train.agg(F.countDistinct(i))
    distinct_counts.show()

+-------------+
|count(Gender)|
+-------------+
|            2|
+-------------+

+----------+
|count(Age)|
+----------+
|         7|
+----------+

+-----------------+
|count(Occupation)|
+-----------------+
|               21|
+-----------------+

+--------------------+
|count(City_Category)|
+--------------------+
|                   3|
+--------------------+

+---------------------------------+
|count(Stay_In_Current_City_Years)|
+---------------------------------+
|                                5|
+---------------------------------+

+---------------------+
|count(Marital_Status)|
+---------------------+
|                    2|
+---------------------+

+-------------------------+
|count(Product_Category_1)|
+-------------------------+
|                       12|
+-------------------------+

+-------------------------+
|count(Product_Category_2)|
+-------------------------+
|                       14|
+-------------------------+

+-------------------------+
|count(Product_Category_

##  4. Count category values within each of the columns
### "Gender","Age","City_Category","Stay_In_Current_City_Years","Marital_Status"

In [11]:
categorical_columns = ["Gender","Age","City_Category","Stay_In_Current_City_Years","Marital_Status"]
for i in categorical_columns:
    train.groupBy(i).agg(F.count(i).alias(i +"_count")).orderBy(i, ascending = False).show(n = 50)

+------+------------+
|Gender|Gender_count|
+------+------------+
|     M|      129227|
|     F|       37594|
+------+------------+

+-----+---------+
|  Age|Age_count|
+-----+---------+
|  55+|     5865|
|51-55|    11166|
|46-50|    13374|
|36-45|    33285|
|26-35|    66942|
|18-25|    31316|
| 0-17|     4873|
+-----+---------+

+-------------+-------------------+
|City_Category|City_Category_count|
+-------------+-------------------+
|            C|              56059|
|            B|              69243|
|            A|              41519|
+-------------+-------------------+

+--------------------------+--------------------------------+
|Stay_In_Current_City_Years|Stay_In_Current_City_Years_count|
+--------------------------+--------------------------------+
|                        4+|                           25362|
|                         3|                           29268|
|                         2|                           31515|
|                         1|               

##  5. Calculate average Purchase within each of the columns
### "Gender","Age","City_Category","Stay_In_Current_City_Years","Marital_Status"

In [12]:
for i in categorical_columns:
    train.groupBy(i).agg(F.round(F.sum("Purchase")/F.count(i),2).alias("Avg_Purchase_"+i)).orderBy(i, ascending = False).show(n = 50)

+------+-------------------+
|Gender|Avg_Purchase_Gender|
+------+-------------------+
|     M|           11824.92|
|     F|           11084.72|
+------+-------------------+

+-----+----------------+
|  Age|Avg_Purchase_Age|
+-----+----------------+
|  55+|        11861.52|
|51-55|         12035.5|
|46-50|        11663.98|
|36-45|        11729.36|
|26-35|        11612.25|
|18-25|        11580.86|
| 0-17|        11172.36|
+-----+----------------+

+-------------+--------------------------+
|City_Category|Avg_Purchase_City_Category|
+-------------+--------------------------+
|            C|                  12207.52|
|            B|                  11488.09|
|            A|                  11199.87|
+-------------+--------------------------+

+--------------------------+---------------------------------------+
|Stay_In_Current_City_Years|Avg_Purchase_Stay_In_Current_City_Years|
+--------------------------+---------------------------------------+
|                        4+|            

##  6. Label encode the following columns
### Age, Gender, Stay_In_Current_City_Years, City_Category

In [13]:
SI_Age = StringIndexer(inputCol="Age",outputCol="Age_le")
SI_Gender = StringIndexer(inputCol="Gender",outputCol="Gender_le")
SI_City_Years = StringIndexer(inputCol="Stay_In_Current_City_Years",outputCol="Stay_In_Current_City_Years_le")
SI_City_Category= StringIndexer(inputCol="City_Category",outputCol="City_Category_le")

In [14]:
train.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



In [15]:
SI_Age_OBJ = SI_Age.fit(train)
SI_Gender_OBJ = SI_Gender.fit(train)
SI_City_Years_OBJ = SI_City_Years.fit(train)
SI_City_Category_OBJ = SI_City_Category.fit(train) 

In [16]:
train_encoded = SI_Age_OBJ.transform(train)
train_encoded = SI_Gender_OBJ.transform(train_encoded)
train_encoded = SI_City_Years_OBJ.transform(train_encoded)
train_encoded = SI_City_Category_OBJ.transform(train_encoded)

##  7. One-Hot encode following columns:
### Gender, City_Category, Occupation

In [17]:
OHE_cat = OneHotEncoder(inputCols = ["Gender_le","City_Category_le","Occupation"],outputCols=["Gender_ohe","City_Category_ohe","Occupation_ohe"])

In [18]:
OHE_cat_OBJ = OHE_cat.fit(train_encoded)

In [19]:
train_encoded = OHE_cat_OBJ.transform(train_encoded)

In [20]:
selected_features = ["Gender_ohe", "Age_le", "Occupation_ohe", "City_Category_le","Stay_In_Current_City_Years_le","Marital_Status","Product_Category_1","Product_Category_2","Product_Category_3"]

In [21]:
assembler = VectorAssembler(inputCols= selected_features, outputCol = "feature_vector")

In [22]:
train_encoded = assembler.transform(train_encoded)

In [23]:
train_encoded.select("feature_vector").show()

+--------------------+
|      feature_vector|
+--------------------+
|(28,[1,12,22,23,2...|
|(28,[0,1,9,23,24,...|
|(28,[0,22,24,25,2...|
|(28,[1,11,22,25,2...|
|(28,[1,11,22,25,2...|
|(28,[0,1,3,24,25,...|
|(28,[0,14,22,23,2...|
|(28,[0,14,22,23,2...|
|(28,[0,19,22,23,2...|
|(28,[1,3,23,24,25...|
|(28,[1,3,23,24,25...|
|(28,[1,3,23,24,25...|
|(28,[1,3,23,24,25...|
|(28,[1,3,23,24,25...|
|(28,[1,3,23,24,25...|
|(28,[1,3,23,24,25...|
|(28,[1,3,23,24,25...|
|(28,[3,22,25,26,2...|
|(28,[0,1,3,22,23,...|
|(28,[0,9,22,25,26...|
+--------------------+
only showing top 20 rows



## 8. Build a baseline model using any of the ML algorithms - LinearRegressor

In [24]:
model = LinearRegression(featuresCol="feature_vector", labelCol="Purchase")
model_linear = model.fit(train_encoded)

In [25]:
evaluator = RegressionEvaluator(labelCol="Purchase", metricName='r2')
evaluator_mae = RegressionEvaluator(labelCol="Purchase", metricName='mae')
evaluator_mse = RegressionEvaluator(labelCol="Purchase", metricName='mse')
evaluator_rmse = RegressionEvaluator(labelCol="Purchase", metricName='rmse')

In [26]:
evaluator.evaluate(model_linear.transform(train_encoded))

0.16624938335224082

In [27]:
evaluator_mae.evaluate(model_linear.transform(train_encoded))
#3731.194026455879

3731.194026455879

In [28]:
evaluator_mse.evaluate(model_linear.transform(train_encoded))
#21535358.27117582

21535358.27117582

In [29]:
evaluator_rmse.evaluate(model_linear.transform(train_encoded))
#4640.620461875311

4640.620461875311

## 9. Model improvement with Grid-Search CV

In [30]:
model_lr = LinearRegression(featuresCol="feature_vector", labelCol="Purchase")
paramGrid = ParamGridBuilder().addGrid(model_lr.maxIter, [10, 50, 100]) \
                              .addGrid(model_lr.regParam, [0.1, 0.01, 0.001]) \
                              .addGrid(model_lr.elasticNetParam, [0.0, 0.5, 1.0]) \
                              .build()
cv = CrossValidator(estimator = model_lr,
                   estimatorParamMaps=paramGrid,
                   evaluator = evaluator,
                   numFolds = 5,
                   seed = 123)

In [31]:
grid_model = cv.fit(train_encoded)

In [32]:
param_dict = grid_model.bestModel.extractParamMap()

In [33]:
final_dict = {}
for k, v in param_dict.items():
    final_dict[k.name] = v

In [34]:
final_dict

{'aggregationDepth': 2,
 'elasticNetParam': 1.0,
 'epsilon': 1.35,
 'featuresCol': 'feature_vector',
 'fitIntercept': True,
 'labelCol': 'Purchase',
 'loss': 'squaredError',
 'maxBlockSizeInMB': 0.0,
 'maxIter': 50,
 'predictionCol': 'prediction',
 'regParam': 0.1,
 'solver': 'auto',
 'standardization': True,
 'tol': 1e-06}

In [35]:
evaluator.evaluate(grid_model.transform(train_encoded))

0.16624929626104124

## 10.Create a Spark ML Pipeline for the final model.
### Assignment_MLPipeline.ipynb